# TileDB Arrays

In [2]:

from pathlib import Path
import shutil
import imageio.v3 as iio
import numpy as np
from numpy.typing import ArrayLike
import tiledb
from PIL import Image

In [3]:
# Read image and convert to TileDB Array
src_path = Path().absolute().joinpath("data/reco100.tiff").__str__()
volume: ArrayLike = iio.imread(src_path)
volume.shape, volume.dtype

((100, 2016, 2016), dtype('uint8'))

In [4]:
# Use a compressor
gzip_filter = tiledb.GzipFilter()

# Create domain specifying dimensions
domain = tiledb.Domain([
    tiledb.Dim(name="x", domain=(0, volume.shape[0] - 1), tile=1, dtype=np.uint16, filters=[gzip_filter]),
    tiledb.Dim(name="y", domain=(0, volume.shape[1] - 1), tile=1, dtype=np.uint16, filters=[gzip_filter]),
    tiledb.Dim(name="z", domain=(0, volume.shape[2] - 1), tile=1, dtype=np.uint16, filters=[gzip_filter])
])

attr = tiledb.Attr(name="volume", dtype=np.uint16)

# Create dense array schema
schema = tiledb.ArraySchema(
    domain=domain,
    attrs=[attr],
    cell_order="row-major",
    tile_order="row-major",
    sparse=False
)

# # Array location on disk
data_path = Path().absolute().joinpath("data/arr").__str__()
if tiledb.array_exists(uri=data_path, isdense=True):
    shutil.rmtree(data_path)
tiledb.DenseArray.create(data_path, schema=schema)

In [5]:
# Write data tp array
with tiledb.open(data_path, mode='r') as arr:
    print(arr.shape)
    print(arr[:10, :10, :10])

(100, 2016, 2016)
OrderedDict([('volume', array([[[65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535],
        [65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
         65535]],

       [[65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 6

In [6]:
with tiledb.open(data_path, mode='w') as arr:
    arr[:1, :arr.shape[1], :arr.shape[2]] = volume[0].astype(np.uint16)

In [ ]:
with tiledb.open(data_path, "r") as arr:
    # Fetch the entire 2D array
    image_data = arr[0, :, :]
    img = Image.fromarray(image_data["volume"])
img

NameError: name 'tiledb' is not defined